# Simulations

In this notebook, we perform ten albedo management and afforestation simulations with pre-trained machine learning models. We compute OLS mappings of median income to simulated UHI to compare the coefficient before and after the simulation. Further, we calculate the gap between low and high income groups at the urban level across each scenario.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
from xgboost import XGBRegressor
import joblib
from statsmodels.stats.weightstats import DescrStatsW

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Import and Processing + Model Import

In [2]:
data = pd.read_csv('data/data.csv', low_memory=False)
urban_df = data[['Urban_name', 'Climate Zone', 'Coastal?']].drop_duplicates().copy()

data['Coastal?'] = data['Coastal?'].fillna(False).astype(int)

mapping_dict = {'Arid': 0, 'Snow': 1, 'Temperate': 2, 'Tropical': 3}
data['Climate Zone'] = data['Climate Zone'].map(mapping_dict)

In [3]:
model_cuhi_d = joblib.load('models/full/CUHI_day_summer_xgb_estimator.joblib')
model_cuhi_n = joblib.load('models/full/CUHI_night_summer_xgb_estimator.joblib')
model_suhi_d = joblib.load('models/full/SUHI_day_summer_xgb_estimator.joblib')
model_suhi_n = joblib.load('models/full/SUHI_night_summer_xgb_estimator.joblib')

In [4]:
features = ['Built Fraction', 'Grass Fraction', 'Tree Fraction', 
            'Built Albedo', 'Grass Albedo', 'Tree Albedo', 
            'Elevation', 'Climate Zone', 'Coastal?']
labels   = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']

### Median Income Columns (Full Dataset)

In [5]:
cols = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']

med25 = 'Median Income (25th percentile)'
med75 = 'Median Income (75th percentile)'
# Calculate the 25th and 75th percentile of median income by urban area
grouped25 = pd.DataFrame({med25: data.groupby(['Urban_name'])['Median Income'].quantile(0.25)}).reset_index()
grouped75 = pd.DataFrame({med75: data.groupby(['Urban_name'])['Median Income'].quantile(0.75)}).reset_index()
# Add a column for the 25th and 75th percentile of median income by urban area
data = data.merge(grouped25, on='Urban_name')
data = data.merge(grouped75, on='Urban_name')
# Compare tract median income with urban area 25th and 75th percentile median income
data[med25 + '?'] = data['Median Income'] <= data[med25]
data[med75 + '?'] = data['Median Income'] >= data[med75]

## Simulation Helper Functions

In [6]:
def get_sim_results(X_test): 
    y_pred_cuhi_d = model_cuhi_d.predict(X_test[features])
    y_pred_cuhi_n = model_cuhi_n.predict(X_test[features])
    y_pred_suhi_d = model_suhi_d.predict(X_test[features])
    y_pred_suhi_n = model_suhi_n.predict(X_test[features])
    X_test['y_pred_cuhi_d'] = y_pred_cuhi_d
    X_test['y_pred_cuhi_n'] = y_pred_cuhi_n
    X_test['y_pred_suhi_d'] = y_pred_suhi_d
    X_test['y_pred_suhi_n'] = y_pred_suhi_n
    return X_test

In [7]:
def apply_regression(group, simname):
    preds = ['y_pred_cuhi_d', 'y_pred_cuhi_n', 'y_pred_suhi_d', 'y_pred_suhi_n']
    
    series_dict = {}
    for i in range(len(preds)):
        groupXy = group[['Median Income (K)', preds[i]]].dropna().copy()
        X = groupXy['Median Income (K)']
        y = groupXy[preds[i]]
        X = sm.add_constant(X)
        model   = sm.OLS(y, np.asarray(X))
        results = model.fit()
        series_dict[simname + ' ' + labels[i] + ' Slope'] = results.params['x1']
        series_dict[simname + ' ' + labels[i] + ' pval']  = results.pvalues['x1']
        series_dict[simname + ' ' + labels[i] + ' SE']    = results.bse['x1']
    
    return pd.Series(series_dict)

In [8]:
def get_ols_results(sim_results, simname):
    ols_result = sim_results.groupby('Urban_name').apply(lambda group: apply_regression(group, simname))
    ols_result.reset_index(inplace=True)
    return ols_result

In [9]:
def apply_quantile(df, simname):
    def weighted_average(group, x):
        return np.average(group[x], weights=group['Area']) 
    def weighted_std(group, x):
        return DescrStatsW(group[x], weights=group['Area']).std
    
    labels  = ['y_pred_cuhi_d', 'y_pred_cuhi_n', 'y_pred_suhi_d', 'y_pred_suhi_n']
    cols    = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']

    med25 = 'Median Income (25th percentile)'
    med75 = 'Median Income (75th percentile)'

    iqr_df = pd.DataFrame(urban_df['Urban_name'])
    for i in range(len(cols)):    
        # Calculate the overall AWM by urban area
        col_awm = simname + ' ' + cols[i] + ' AWM'
        awm = pd.DataFrame({col_awm: df[['Urban_name', labels[i], 'Area']].dropna().groupby(['Urban_name']).apply(lambda group: weighted_average(group, x=labels[i]))}).reset_index()  
        iqr_df = iqr_df.merge(awm, on='Urban_name')
        # Filter data points at or below Q1 and Q3, and then calculate the median predicted UHI by urban area
        col25 = simname + ' ' + cols[i] + ' (25th %tile)'
        col75 = simname + ' ' + cols[i] + ' (75th %tile)'
        m25 = pd.DataFrame({col25: df[df[med25 + '?'] == True][['Urban_name', labels[i], 'Area']].dropna().groupby(['Urban_name']).apply(lambda group: weighted_average(group, x=labels[i])), 
                            col25 + ' Std': df[df[med25 + '?'] == True][['Urban_name', labels[i], 'Area']].dropna().groupby(['Urban_name']).apply(lambda group: weighted_std(group, x=labels[i]))}).reset_index()  
        m75 = pd.DataFrame({col75: df[df[med75 + '?'] == True][['Urban_name', labels[i], 'Area']].dropna().groupby(['Urban_name']).apply(lambda group: weighted_average(group, x=labels[i])), 
                            col75 + ' Std': df[df[med75 + '?'] == True][['Urban_name', labels[i], 'Area']].dropna().groupby(['Urban_name']).apply(lambda group: weighted_std(group, x=labels[i]))}).reset_index()
        # Update the results
        iqr_df = iqr_df.merge(m25, on='Urban_name')
        iqr_df = iqr_df.merge(m75, on='Urban_name')
        iqr_df[simname + ' ' + cols[i] + ' (Difference)']  = iqr_df[col75] - iqr_df[col25]
    return iqr_df

In [10]:
def run_simulation(albedo, afforest, afforest_pct, lowincomeonly, simname, filename):
    X = data.copy().dropna(subset=features+labels)
    
    if albedo == True:
        if lowincomeonly == True:
            X['Built Albedo'] = np.where(X['Median Income (25th percentile)?'] == True, (0.8 - X['Albedo_Built_rur']), X['Built Albedo'])
        else:
            X.loc[:, 'Built Albedo'] = 0.8 - X['Albedo_Built_rur']
    if afforest == True:  
        if lowincomeonly == True:
            X['Tree Fraction']  = np.where(X['Median Income (25th percentile)?'] == True, (X['ESATreeFraction'] + (afforest_pct/100 * X['ESAGrassFraction'])) - X['ESATreeFractionRural'], X['Tree Fraction'])
            X['Grass Fraction'] = np.where(X['Median Income (25th percentile)?'] == True, (X['ESAGrassFraction'] - (afforest_pct/100 * X['ESAGrassFraction'])) - X['ESAGrassFractionRural'], X['Grass Fraction'])
            X['Grass Albedo']   = np.where(X['Median Income (25th percentile)?'] == True, (X['Albedo_Grass'] * X['Tree Fraction'] + X['Albedo_Tree'] * X['Tree Fraction']) - X['Albedo_Grass_rur'], X['Grass Albedo'])

        else: 
            X['Tree Fraction']  = (X['ESATreeFraction'] + (afforest_pct/100 * X['ESAGrassFraction'])) - X['ESATreeFractionRural']
            X['Grass Fraction'] = (X['ESAGrassFraction'] - (afforest_pct/100 * X['ESAGrassFraction'])) - X['ESAGrassFractionRural']
            X['Grass Albedo']   = (X['Albedo_Grass'] * X['Tree Fraction'] + X['Albedo_Tree'] * X['Tree Fraction']) - X['Albedo_Grass_rur']
    
    sim_results = get_sim_results(X)
    sim_results.to_csv('tables/sim_output/{}.csv'.format(filename), index=False)
    iqr_result = apply_quantile(sim_results, simname)
    ols_result = get_ols_results(sim_results, simname)
    return iqr_result, ols_result

- Save label results in separate files for comparison
- Include quartile results and coefficients in same file

In [11]:
def run_all():
    # Simulations
    simnames  = ['Baseline', 'Albedo Management', 'Targeted Albedo Management', 
                 '50% Afforestation', '100% Afforestation', 
                 'Targeted 50% Afforestation', 'Targeted 100% Afforestation', 
                 'Albedo Management/50% Afforestation', 'Albedo Management/100% Afforestation', 
                 'Targeted Albedo Management/50% Afforestation', 'Targeted Albedo Management/100% Afforestation']
    filenames = ['baseline', 'albedo', 'albedo_targeted', 'afforest_50', 'afforest_100', 
                 'afforest50_targeted', 'afforest100_targeted', 
                 'albedo_afforest_50', 'albedo_afforest_100', 
                 'albedo_afforest_50_targeted', 'albedo_afforest_100_targeted']
    albedo_list = [False, True, True, False, False, False, False, True, True, True, True]
    afforest_list = [False, False, False, True, True, True, True, True, True, True, True]
    afforest_pct_list = [0,0,0,50,100,50,100,50,100,50,100]
    lowincomeonly_list = [False, False, True, False, False, True, True, False, False, True, True]
    sim_cond = [ (a, b, c, d) for a, b, c, d in zip(albedo_list, afforest_list, afforest_pct_list, lowincomeonly_list)]
    result = urban_df.copy().reset_index().drop(columns='index')
    for i in range(len(sim_cond)):
        iqr_result, ols_result = run_simulation(albedo=sim_cond[i][0], afforest=sim_cond[i][1], 
                                                afforest_pct=sim_cond[i][2], lowincomeonly=sim_cond[i][3], 
                                                simname=simnames[i], filename=filenames[i])
        result = result.merge(iqr_result, on='Urban_name')
        result = result.merge(ols_result, on='Urban_name')
    result.to_csv('tables/sim_results/sim_results.csv', index=False)
    return result

In [12]:
final_result = run_all()

In [13]:
len(final_result)

493

In [14]:
final_result.head()

,Urban_name,Climate Zone,Coastal?,Baseline CUHI Day AWM,Baseline CUHI Day (25th %tile),Baseline CUHI Day (25th %tile) Std,Baseline CUHI Day (75th %tile),Baseline CUHI Day (75th %tile) Std,Baseline CUHI Day (Difference),Baseline CUHI Night AWM,Baseline CUHI Night (25th %tile),Baseline CUHI Night (25th %tile) Std,Baseline CUHI Night (75th %tile),Baseline CUHI Night (75th %tile) Std,Baseline CUHI Night (Difference),Baseline SUHI Day AWM,Baseline SUHI Day (25th %tile),Baseline SUHI Day (25th %tile) Std,Baseline SUHI Day (75th %tile),Baseline SUHI Day (75th %tile) Std,Baseline SUHI Day (Difference),Baseline SUHI Night AWM,Baseline SUHI Night (25th %tile),Baseline SUHI Night (25th %tile) Std,Baseline SUHI Night (75th %tile),Baseline SUHI Night (75th %tile) Std,Baseline SUHI Night (Difference),Baseline CUHI Day Slope,Baseline CUHI Day pval,Baseline CUHI Day SE,Baseline CUHI Night Slope,Baseline CUHI Night pval,Baseline CUHI Night SE,Baseline SUHI Day Slope,Baseline SUHI Day pval,Baseline SUHI Day SE,Baseline SUHI Night Slope,Baseline SUHI Night pval,Baseline SUHI Night SE,Albedo Management CUHI Day AWM,Albedo Management CUHI Day (25th %tile),Albedo Management CUHI Day (25th %tile) Std,Albedo Management CUHI Day (75th %tile),Albedo Management CUHI Day (75th %tile) Std,Albedo Management CUHI Day (Difference),Albedo Management CUHI Night AWM,Albedo Management CUHI Night (25th %tile),Albedo Management CUHI Night (25th %tile) Std,Albedo Management CUHI Night (75th %tile),Albedo Management CUHI Night (75th %tile) Std,Albedo Management CUHI Night (Difference),Albedo Management SUHI Day AWM,Albedo Management SUHI Day (25th %tile),Albedo Management SUHI Day (25th %tile) Std,Albedo Management SUHI Day (75th %tile),Albedo Management SUHI Day (75th %tile) Std,Albedo Management SUHI Day (Difference),Albedo Management SUHI Night AWM,Albedo Management SUHI Night (25th %tile),Albedo Management SUHI Night (25th %tile) Std,Albedo Management SUHI Night (75th %tile),Albedo Management SUHI Night (75th %tile) Std,Albedo Management SUHI Night (Difference),Albedo Management CUHI Day Slope,Albedo Management CUHI Day pval,Albedo Management CUHI Day SE,Albedo Management CUHI Night Slope,Albedo Management CUHI Night pval,Albedo Management CUHI Night SE,Albedo Management SUHI Day Slope,Albedo Management SUHI Day pval,Albedo Management SUHI Day SE,Albedo Management SUHI Night Slope,Albedo Management SUHI Night pval,Albedo Management SUHI Night SE,Targeted Albedo Management CUHI Day AWM,Targeted Albedo Management CUHI Day (25th %tile),Targeted Albedo Management CUHI Day (25th %tile) Std,Targeted Albedo Management CUHI Day (75th %tile),Targeted Albedo Management CUHI Day (75th %tile) Std,Targeted Albedo Management CUHI Day (Difference),Targeted Albedo Management CUHI Night AWM,Targeted Albedo Management CUHI Night (25th %tile),Targeted Albedo Management CUHI Night (25th %tile) Std,Targeted Albedo Management CUHI Night (75th %tile),Targeted Albedo Management CUHI Night (75th %tile) Std,Targeted Albedo Management CUHI Night (Difference),Targeted Albedo Management SUHI Day AWM,Targeted Albedo Management SUHI Day (25th %tile),Targeted Albedo Management SUHI Day (25th %tile) Std,Targeted Albedo Management SUHI Day (75th %tile),Targeted Albedo Management SUHI Day (75th %tile) Std,Targeted Albedo Management SUHI Day (Difference),Targeted Albedo Management SUHI Night AWM,Targeted Albedo Management SUHI Night (25th %tile),Targeted Albedo Management SUHI Night (25th %tile) Std,Targeted Albedo Management SUHI Night (75th %tile),Targeted Albedo Management SUHI Night (75th %tile) Std,Targeted Albedo Management SUHI Night (Difference),Targeted Albedo Management CUHI Day Slope,Targeted Albedo Management CUHI Day pval,Targeted Albedo Management CUHI Day SE,Targeted Albedo Management CUHI Night Slope,Targeted Albedo Management CUHI Night pval,Targeted Albedo Management CUHI Night SE,Targeted Albedo Management SUHI Day Slope,Targeted Albedo Management SUHI Day pval,Targeted Albedo M

In [15]:
final_result['Baseline CUHI Day (25th %tile)'].mean()

0.10848036366621229

## Results

### Predicted Urban Heat Intensity by First and Third Quartile of Median Income

In [16]:
sims = ['Baseline', 'Albedo Management', 'Targeted Albedo Management', 
                 '50% Afforestation', '100% Afforestation', 
                 'Targeted 50% Afforestation', 'Targeted 100% Afforestation', 
                 'Albedo Management/50% Afforestation', 'Albedo Management/100% Afforestation', 
                 'Targeted Albedo Management/50% Afforestation', 'Targeted Albedo Management/100% Afforestation']

In [17]:
vars1  = [x + ' CUHI Day (Difference)' for x in sims]

In [18]:
final_result[[x + ' CUHI Day (Difference)' for x in sims]].median().round(2)

Baseline CUHI Day (Difference)                                        -0.29
Albedo Management CUHI Day (Difference)                               -0.29
Targeted Albedo Management CUHI Day (Difference)                      -0.24
50% Afforestation CUHI Day (Difference)                               -0.31
100% Afforestation CUHI Day (Difference)                              -0.35
Targeted 50% Afforestation CUHI Day (Difference)                      -0.07
Targeted 100% Afforestation CUHI Day (Difference)                      0.06
Albedo Management/50% Afforestation CUHI Day (Difference)             -0.32
Albedo Management/100% Afforestation CUHI Day (Difference)            -0.38
Targeted Albedo Management/50% Afforestation CUHI Day (Difference)     0.04
Targeted Albedo Management/100% Afforestation CUHI Day (Difference)    0.12
dtype: float64

In [19]:
mean_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].mean().tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].mean().tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].mean().tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].mean().tolist()})

In [20]:
mean_gap.round(2)

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,-0.49,-0.38,-2.34,-0.71
1,Albedo Management,-0.47,-0.33,-2.06,-0.55
2,Targeted Albedo Management,-0.44,-0.29,-1.75,-0.42
3,50% Afforestation,-0.52,-0.20,-2.49,-0.49
4,100% Afforestation,-0.59,-0.27,-2.52,-0.48
5,Targeted 50% Afforestation,-0.22,-0.51,-1.08,-0.94
6,Targeted 100% Afforestation,-0.09,-0.56,-0.37,-1.03
7,Albedo Management/50% Afforestation,-0.51,-0.19,-2.46,-0.42
8,Albedo Management/100% Afforestation,-0.58,-0.27,-2.37,-0.40
9,Targeted Albedo Management/50% Afforestation,-0.13,-0.52,-0.56,-0.79


In [21]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Day (Difference)' for x in sims]].mean()).T.round(2)

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Day (Difference),-0.62,-0.65,-0.43,0.16
Albedo Management CUHI Day (Difference),-0.58,-0.59,-0.42,0.17
Targeted Albedo Management CUHI Day (Difference),-0.54,-0.57,-0.39,0.16
50% Afforestation CUHI Day (Difference),-0.66,-0.65,-0.46,0.15
100% Afforestation CUHI Day (Difference),-0.70,-0.72,-0.53,-0.08
Targeted 50% Afforestation CUHI Day (Difference),-0.30,-0.41,-0.16,0.44
Targeted 100% Afforestation CUHI Day (Difference),-0.15,-0.29,-0.02,0.53
Albedo Management/50% Afforestation CUHI Day (Difference),-0.62,-0.62,-0.47,0.21
Albedo Management/100% Afforestation CUHI Day (Difference),-0.66,-0.74,-0.52,0.03
Targeted Albedo Management/50% Afforestation CUHI Day (Difference),-0.17,-0.31,-0.07,0.52


In [73]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Day (Difference)' for x in sims]].mean()).T.round(2).to_csv('tables/sim_gap_cuhi_day_clim_zone.csv')


In [22]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Night (Difference)' for x in sims]].mean()).T.round(2)

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Night (Difference),-0.53,-0.56,-0.31,0.21
Albedo Management CUHI Night (Difference),-0.56,-0.49,-0.25,0.27
Targeted Albedo Management CUHI Night (Difference),-0.52,-0.44,-0.21,0.29
50% Afforestation CUHI Night (Difference),-0.49,-0.32,-0.12,0.20
100% Afforestation CUHI Night (Difference),-0.53,-0.40,-0.19,0.27
Targeted 50% Afforestation CUHI Night (Difference),-0.80,-0.71,-0.41,0.22
Targeted 100% Afforestation CUHI Night (Difference),-0.76,-0.80,-0.45,0.21
Albedo Management/50% Afforestation CUHI Night (Difference),-0.46,-0.33,-0.10,0.26
Albedo Management/100% Afforestation CUHI Night (Difference),-0.56,-0.43,-0.19,0.34
Targeted Albedo Management/50% Afforestation CUHI Night (Difference),-0.90,-0.71,-0.41,0.18


In [74]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Night (Difference)' for x in sims]].mean()).T.round(2).to_csv('tables/sim_gap_cuhi_night_clim_zone.csv')

In [23]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Day (Difference)' for x in sims]].mean()).T.round(2)

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Day (Difference),-0.93,-3.61,-2.10,0.97
Albedo Management SUHI Day (Difference),-0.79,-3.09,-1.92,1.28
Targeted Albedo Management SUHI Day (Difference),-0.33,-2.87,-1.58,1.57
50% Afforestation SUHI Day (Difference),-1.68,-3.76,-2.19,1.14
100% Afforestation SUHI Day (Difference),-1.78,-3.69,-2.24,0.74
Targeted 50% Afforestation SUHI Day (Difference),0.33,-2.63,-0.71,1.89
Targeted 100% Afforestation SUHI Day (Difference),0.76,-1.96,0.08,2.41
Albedo Management/50% Afforestation SUHI Day (Difference),-1.74,-3.71,-2.14,1.02
Albedo Management/100% Afforestation SUHI Day (Difference),-1.72,-3.44,-2.11,0.95
Targeted Albedo Management/50% Afforestation SUHI Day (Difference),0.73,-1.95,-0.22,1.98


In [76]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Day (Difference)' for x in sims]].mean()).T.round(2).to_csv('tables/sim_gap_suhi_day_clim_zone.csv')


In [24]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Night (Difference)' for x in sims]].mean()).T.round(2)

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Night (Difference),-0.66,-1.11,-0.57,0.39
Albedo Management SUHI Night (Difference),-0.51,-0.82,-0.47,0.36
Targeted Albedo Management SUHI Night (Difference),-0.46,-0.72,-0.32,0.39
50% Afforestation SUHI Night (Difference),-0.64,-0.77,-0.36,0.14
100% Afforestation SUHI Night (Difference),-0.60,-0.74,-0.37,0.17
Targeted 50% Afforestation SUHI Night (Difference),-1.09,-1.35,-0.78,0.26
Targeted 100% Afforestation SUHI Night (Difference),-1.24,-1.52,-0.83,0.21
Albedo Management/50% Afforestation SUHI Night (Difference),-0.55,-0.65,-0.32,0.09
Albedo Management/100% Afforestation SUHI Night (Difference),-0.51,-0.58,-0.32,0.19
Targeted Albedo Management/50% Afforestation SUHI Night (Difference),-0.80,-1.19,-0.65,0.21


In [75]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Night (Difference)' for x in sims]].mean()).T.round(2).to_csv('tables/sim_gap_suhi_night_clim_zone.csv')

In [25]:
mean_slope = pd.DataFrame({'Simulation':sims, 
                             'CUHI Day':final_result[[x + ' CUHI Day Slope' for x in sims]].mean().tolist(), 
                             'CUHI Night':final_result[[x + ' CUHI Night Slope' for x in sims]].mean().tolist(),
                             'SUHI Day':final_result[[x + ' SUHI Day Slope' for x in sims]].mean().tolist(),
                             'SUHI Night':final_result[[x + ' SUHI Night Slope' for x in sims]].mean().tolist()}).round(3)

In [26]:
mean_slope

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,-0.014,-0.011,-0.085,-0.022
1,Albedo Management,-0.013,-0.009,-0.078,-0.018
2,Targeted Albedo Management,-0.012,-0.007,-0.064,-0.011
3,50% Afforestation,-0.014,-0.005,-0.086,-0.015
4,100% Afforestation,-0.017,-0.006,-0.085,-0.014
5,Targeted 50% Afforestation,-0.007,-0.012,-0.050,-0.025
6,Targeted 100% Afforestation,-0.004,-0.014,-0.031,-0.027
7,Albedo Management/50% Afforestation,-0.015,-0.004,-0.084,-0.014
8,Albedo Management/100% Afforestation,-0.017,-0.005,-0.078,-0.011
9,Targeted Albedo Management/50% Afforestation,-0.004,-0.012,-0.032,-0.019


In [27]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Day Slope' for x in sims]].mean()).round(3).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Day Slope,-0.015,-0.017,-0.013,0.009
Albedo Management CUHI Day Slope,-0.013,-0.016,-0.013,0.011
Targeted Albedo Management CUHI Day Slope,-0.011,-0.014,-0.012,0.008
50% Afforestation CUHI Day Slope,-0.014,-0.018,-0.014,0.008
100% Afforestation CUHI Day Slope,-0.014,-0.021,-0.016,0.004
Targeted 50% Afforestation CUHI Day Slope,-0.007,-0.011,-0.006,0.028
Targeted 100% Afforestation CUHI Day Slope,-0.006,-0.008,-0.003,0.040
Albedo Management/50% Afforestation CUHI Day Slope,-0.014,-0.019,-0.014,0.012
Albedo Management/100% Afforestation CUHI Day Slope,-0.015,-0.022,-0.016,0.008
Targeted Albedo Management/50% Afforestation CUHI Day Slope,-0.003,-0.008,-0.003,0.032


In [28]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Night Slope' for x in sims]].mean()).round(3).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Night Slope,-0.012,-0.016,-0.010,0.017
Albedo Management CUHI Night Slope,-0.011,-0.013,-0.008,0.019
Targeted Albedo Management CUHI Night Slope,-0.009,-0.011,-0.006,0.022
50% Afforestation CUHI Night Slope,-0.005,-0.008,-0.004,0.024
100% Afforestation CUHI Night Slope,-0.006,-0.009,-0.006,0.025
Targeted 50% Afforestation CUHI Night Slope,-0.013,-0.019,-0.011,0.022
Targeted 100% Afforestation CUHI Night Slope,-0.012,-0.021,-0.012,0.026
Albedo Management/50% Afforestation CUHI Night Slope,-0.003,-0.007,-0.004,0.023
Albedo Management/100% Afforestation CUHI Night Slope,-0.005,-0.008,-0.006,0.025
Targeted Albedo Management/50% Afforestation CUHI Night Slope,-0.013,-0.018,-0.010,0.019


In [29]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Day Slope' for x in sims]].mean()).round(3).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Day Slope,-0.039,-0.122,-0.082,0.051
Albedo Management SUHI Day Slope,-0.035,-0.108,-0.077,0.080
Targeted Albedo Management SUHI Day Slope,-0.023,-0.095,-0.062,0.087
50% Afforestation SUHI Day Slope,-0.046,-0.128,-0.080,0.090
100% Afforestation SUHI Day Slope,-0.040,-0.129,-0.079,0.069
Targeted 50% Afforestation SUHI Day Slope,-0.017,-0.095,-0.040,0.091
Targeted 100% Afforestation SUHI Day Slope,-0.008,-0.079,-0.019,0.116
Albedo Management/50% Afforestation SUHI Day Slope,-0.044,-0.128,-0.077,0.089
Albedo Management/100% Afforestation SUHI Day Slope,-0.041,-0.119,-0.072,0.087
Targeted Albedo Management/50% Afforestation SUHI Day Slope,-0.004,-0.074,-0.022,0.090


In [30]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Night Slope' for x in sims]].mean()).round(3).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Night Slope,-0.017,-0.035,-0.019,0.029
Albedo Management SUHI Night Slope,-0.013,-0.027,-0.016,0.030
Targeted Albedo Management SUHI Night Slope,-0.007,-0.020,-0.009,0.031
50% Afforestation SUHI Night Slope,-0.012,-0.024,-0.012,0.011
100% Afforestation SUHI Night Slope,-0.012,-0.021,-0.012,0.013
Targeted 50% Afforestation SUHI Night Slope,-0.021,-0.039,-0.021,0.019
Targeted 100% Afforestation SUHI Night Slope,-0.024,-0.043,-0.022,0.020
Albedo Management/50% Afforestation SUHI Night Slope,-0.011,-0.021,-0.012,0.007
Albedo Management/100% Afforestation SUHI Night Slope,-0.010,-0.017,-0.010,0.010
Targeted Albedo Management/50% Afforestation SUHI Night Slope,-0.011,-0.032,-0.016,0.016


In [31]:
data[labels].describe().round(2)

,CUHI Day,CUHI Night,SUHI Day,SUHI Night
count,55430.00,55430.00,55682.00,55682.00
mean,0.26,0.33,1.97,0.70
std,0.70,0.56,3.00,0.98
min,-9.74,-8.58,-15.99,-7.97
25%,-0.03,0.01,-0.15,0.03
50%,0.23,0.30,1.97,0.74
75%,0.55,0.64,4.09,1.37
max,4.92,2.97,12.75,6.25


In [32]:
min_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].min().tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].min().tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].min().tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].min().tolist()})
min_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,-6.104785,-4.070578,-12.201200,-3.882905
1,Albedo Management,-4.982686,-3.672617,-10.213629,-3.259445
2,Targeted Albedo Management,-6.093557,-3.875567,-11.632141,-3.670652
3,50% Afforestation,-6.459339,-4.747747,-10.990970,-3.409873
4,100% Afforestation,-5.665980,-6.205907,-10.627807,-4.949492
5,Targeted 50% Afforestation,-5.682224,-4.514842,-9.912859,-4.315971
6,Targeted 100% Afforestation,-5.555102,-4.502976,-9.857174,-4.331556
7,Albedo Management/50% Afforestation,-5.133954,-4.674172,-9.015270,-4.103800
8,Albedo Management/100% Afforestation,-5.159994,-7.319930,-9.208220,-3.698526
9,Targeted Albedo Management/50% Afforestation,-5.680891,-4.430189,-9.534442,-4.227752


In [33]:
max_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].max().tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].max().tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].max().tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].max().tolist()})
max_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,2.671853,2.396248,6.238594,2.697962
1,Albedo Management,2.819622,2.116850,6.757996,2.339068
2,Targeted Albedo Management,2.687207,2.127582,6.548720,2.534101
3,50% Afforestation,2.057531,2.324100,6.116336,2.116695
4,100% Afforestation,2.132033,2.182005,3.693329,1.337824
5,Targeted 50% Afforestation,3.021511,2.442365,7.744484,2.136658
6,Targeted 100% Afforestation,3.436636,2.248178,11.887223,1.434425
7,Albedo Management/50% Afforestation,2.261020,2.219023,5.879938,1.939668
8,Albedo Management/100% Afforestation,2.474313,2.175420,3.288635,1.180926
9,Targeted Albedo Management/50% Afforestation,3.083366,2.322087,8.160445,2.257313


In [34]:
q1st_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].quantile(q=0.01).tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].quantile(q=0.01).tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].quantile(q=0.01).tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].quantile(q=0.01).tolist()})
q1st_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,-4.526242,-2.666769,-8.975404,-2.883638
1,Albedo Management,-4.183493,-2.704621,-8.031261,-2.412602
2,Targeted Albedo Management,-4.434553,-2.507263,-8.351927,-2.468639
3,50% Afforestation,-4.750061,-3.151557,-9.212197,-2.773374
4,100% Afforestation,-4.850003,-4.589769,-9.291305,-2.928094
5,Targeted 50% Afforestation,-4.249663,-2.430979,-7.571678,-2.854525
6,Targeted 100% Afforestation,-4.188970,-2.519711,-7.744408,-3.066678
7,Albedo Management/50% Afforestation,-4.225228,-3.148553,-7.970596,-2.202736
8,Albedo Management/100% Afforestation,-4.514281,-4.725109,-8.061100,-2.377539
9,Targeted Albedo Management/50% Afforestation,-4.145223,-2.671685,-7.049418,-2.653734


In [35]:
q99th_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].quantile(q=0.99).tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].quantile(q=0.99).tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].quantile(q=0.99).tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].quantile(q=0.99).tolist()})
q99th_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,1.117868,1.100347,4.097732,1.522948
1,Albedo Management,1.006961,1.073895,3.950152,1.403053
2,Targeted Albedo Management,1.098805,1.048753,4.142602,1.751659
3,50% Afforestation,1.122456,1.094227,2.707015,0.972481
4,100% Afforestation,1.047121,1.018559,2.448422,1.025334
5,Targeted 50% Afforestation,1.372841,0.894147,5.759601,1.251829
6,Targeted 100% Afforestation,1.521234,0.974422,5.525229,1.074510
7,Albedo Management/50% Afforestation,1.265326,1.092702,2.263775,1.004992
8,Albedo Management/100% Afforestation,0.991136,1.105901,2.338667,1.005598
9,Targeted Albedo Management/50% Afforestation,1.570884,0.763886,5.730511,1.361623


In [36]:
q5th_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].quantile(q=0.05).tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].quantile(q=0.05).tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].quantile(q=0.05).tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].quantile(q=0.05).tolist()})
q5th_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,-2.338358,-1.434964,-6.596238,-2.025332
1,Albedo Management,-2.165596,-1.227672,-5.635461,-1.572508
2,Targeted Albedo Management,-2.296084,-1.309648,-5.437835,-1.544648
3,50% Afforestation,-2.488205,-1.195539,-6.893441,-1.538219
4,100% Afforestation,-2.731415,-1.445890,-6.864077,-1.645170
5,Targeted 50% Afforestation,-1.821451,-1.580692,-5.187045,-2.129654
6,Targeted 100% Afforestation,-1.741409,-1.510424,-4.726891,-2.264661
7,Albedo Management/50% Afforestation,-2.173879,-1.186087,-6.434110,-1.418063
8,Albedo Management/100% Afforestation,-2.475375,-1.435109,-6.330772,-1.445936
9,Targeted Albedo Management/50% Afforestation,-1.751029,-1.545472,-4.746796,-1.974522


In [37]:
q95th_gap = pd.DataFrame({'Simulation':sims, 
              'CUHI Day':final_result[[x + ' CUHI Day (Difference)' for x in sims]].quantile(q=0.95).tolist(), 
              'CUHI Night':final_result[[x + ' CUHI Night (Difference)' for x in sims]].quantile(q=0.95).tolist(),
              'SUHI Day':final_result[[x + ' SUHI Day (Difference)' for x in sims]].quantile(q=0.95).tolist(),
              'SUHI Night':final_result[[x + ' SUHI Night (Difference)' for x in sims]].quantile(q=0.95).tolist()})
q95th_gap

,Simulation,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Baseline,0.372449,0.431795,1.282210,0.793364
1,Albedo Management,0.343481,0.393378,1.454358,0.653057
2,Targeted Albedo Management,0.392893,0.451382,1.853331,0.889182
3,50% Afforestation,0.315158,0.544906,1.227246,0.580109
4,100% Afforestation,0.303493,0.505423,1.160718,0.516494
5,Targeted 50% Afforestation,0.742915,0.194665,3.114260,0.388771
6,Targeted 100% Afforestation,0.834203,0.339034,3.542028,0.322900
7,Albedo Management/50% Afforestation,0.318824,0.583516,1.246877,0.562477
8,Albedo Management/100% Afforestation,0.286804,0.586488,1.155456,0.473050
9,Targeted Albedo Management/50% Afforestation,0.838903,0.186087,3.296043,0.532075


In [38]:
pval_df = pd.DataFrame({'Simulations': sims[1:]})
for label in labels:
    n_pvals = []
    for i in range(1,len(sims)):
        col = sims[i] + ' ' + label + ' pval'
        n_pvals.append(len(final_result[final_result[col] < 0.05]))
    pval_df[label] = n_pvals

In [39]:
pval_df

,Simulations,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Albedo Management,319,269,334,290
1,Targeted Albedo Management,285,239,288,198
2,50% Afforestation,265,162,331,245
3,100% Afforestation,264,160,299,222
4,Targeted 50% Afforestation,204,320,246,318
5,Targeted 100% Afforestation,176,337,195,328
6,Albedo Management/50% Afforestation,266,151,331,239
7,Albedo Management/100% Afforestation,265,152,301,210
8,Targeted Albedo Management/50% Afforestation,182,308,186,268
9,Targeted Albedo Management/100% Afforestation,187,327,151,271


In [40]:
slope_df = pd.DataFrame({'Simulations': sims[1:]})
for label in labels:
    n_slopes = []
    for i in range(1,len(sims)):
        col = sims[i] + ' ' + label + ' Slope'
        n_slopes.append(len(final_result[final_result[col] < 0]))
    slope_df[label] = n_slopes

In [41]:
slope_df

,Simulations,CUHI Day,CUHI Night,SUHI Day,SUHI Night
0,Albedo Management,461,413,465,431
1,Targeted Albedo Management,440,382,433,371
2,50% Afforestation,435,359,461,414
3,100% Afforestation,430,353,447,412
4,Targeted 50% Afforestation,338,438,390,429
5,Targeted 100% Afforestation,288,438,348,436
6,Albedo Management/50% Afforestation,438,349,461,403
7,Albedo Management/100% Afforestation,438,337,448,391
8,Targeted Albedo Management/50% Afforestation,273,431,354,402
9,Targeted Albedo Management/100% Afforestation,240,426,304,396


In [42]:
len(final_result[final_result['Albedo Management CUHI Day pval'] < 0.05])

319

In [43]:
len(final_result[final_result['Albedo Management CUHI Day Slope'] < 0])

461

## Two Sample T-test

In [44]:
# Code adapted from: https://365datascience.com/calculators/difference-in-means-calculator/
from scipy.stats import t
def two_sample_ttest(n1, n2, mean1, mean2, std_dev1, std_dev2):
    # Null hypothesis: The mean difference is zero
    # Alternative hypothesis: The mean difference is not zero
    null_mean = 0

    # Calculate the pooled standard deviation
    pooled_std_dev = np.sqrt(((n1-1) * std_dev1**2 + (n2-1) * std_dev2**2) / (n1 + n2 - 2))

    # Calculate the standard error of the difference
    se = pooled_std_dev * np.sqrt((1/n1) + (1/n2))

    # Calculate the t-statistic
    t_statistic = (mean1 - mean2 - null_mean) / se

    # Degrees of freedom
    df = n1 + n2 - 2

    # Calculate the p-value
    p_value = 2 * (1 - t.cdf(abs(t_statistic), df=df))

    # Return the p-value
    return p_value

In [51]:
sims = ['Baseline', 'Albedo Management', 'Targeted Albedo Management', 
        '50% Afforestation', '100% Afforestation', 'Targeted 50% Afforestation', 'Targeted 100% Afforestation', 
        'Albedo Management/50% Afforestation', 'Albedo Management/100% Afforestation', 
        'Targeted Albedo Management/50% Afforestation', 'Targeted Albedo Management/100% Afforestation']
cols    = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']
gap_sig = []
sim_col = []
for sim in sims:
    for col in cols:
        sim_col.append(sim + ' ' + col)

for sim in sim_col:
    pval = two_sample_ttest(n1=data[data['Median Income (75th percentile)?']==True]['Median Income (75th percentile)?'].value_counts()[0],
                            n2=data[data['Median Income (25th percentile)?']==True]['Median Income (25th percentile)?'].value_counts()[0],
                            mean1=final_result[sim + ' (75th %tile)'].mean(),
                            mean2=final_result[sim + ' (25th %tile)'].mean(),
                            std_dev1=final_result[sim + ' (75th %tile) Std'].mean(), 
                            std_dev2=final_result[sim + ' (25th %tile) Std'].mean())
    gap_sig.append(pval)
len(sim_col)

44

In [53]:
sim_sig = pd.DataFrame({'Simulation': sim_col, 'pval':gap_sig})
sim_sig

,Simulation,pval
0,Baseline CUHI Day,0.000000e+00
1,Baseline CUHI Night,0.000000e+00
2,Baseline SUHI Day,0.000000e+00
3,Baseline SUHI Night,0.000000e+00
4,Albedo Management CUHI Day,0.000000e+00
5,Albedo Management CUHI Night,0.000000e+00
6,Albedo Management SUHI Day,0.000000e+00
7,Albedo Management SUHI Night,0.000000e+00
8,Targeted Albedo Management CUHI Day,0.000000e+00
9,Targeted Albedo Management CUHI Night,0.000000e+00


In [55]:
len(sim_sig[sim_sig['pval'] < 0.05])

44

In [56]:
len(sim_sig[sim_sig['pval'] < 0.0001])

44

In [59]:
sims = ['Baseline', 'Albedo Management', 'Targeted Albedo Management', 
        '50% Afforestation', '100% Afforestation', 'Targeted 50% Afforestation', 'Targeted 100% Afforestation', 
        'Albedo Management/50% Afforestation', 'Albedo Management/100% Afforestation', 
        'Targeted Albedo Management/50% Afforestation', 'Targeted Albedo Management/100% Afforestation']
cols    = ['CUHI Day', 'CUHI Night', 'SUHI Day', 'SUHI Night']
gap_sig_arid = []
gap_sig_snow = []
gap_sig_temp = []
gap_sig_trop = []
sim_col = []
for sim in sims:
    for col in cols:
        sim_col.append(sim + ' ' + col)

#idx+=0
for sim in sim_col:
    pval = two_sample_ttest(n1=data[(data['Median Income (75th percentile)?']==True) & (data['Climate Zone'] == 0)]['Median Income (75th percentile)?'].value_counts()[0],
                            n2=data[(data['Median Income (25th percentile)?']==True) & (data['Climate Zone'] == 0)]['Median Income (25th percentile)?'].value_counts()[0],
                            mean1=final_result[final_result['Climate Zone']=='Arid'][sim + ' (75th %tile)'].mean(),
                            mean2=final_result[final_result['Climate Zone']=='Arid'][sim + ' (25th %tile)'].mean(),
                            std_dev1=final_result[final_result['Climate Zone']=='Arid'][sim + ' (75th %tile) Std'].mean(), 
                            std_dev2=final_result[final_result['Climate Zone']=='Arid'][sim + ' (25th %tile) Std'].mean())
    gap_sig_arid.append(pval)
    
    pval = two_sample_ttest(n1=data[(data['Median Income (75th percentile)?']==True) & (data['Climate Zone'] == 1)]['Median Income (75th percentile)?'].value_counts()[0],
                            n2=data[(data['Median Income (25th percentile)?']==True) & (data['Climate Zone'] == 1)]['Median Income (25th percentile)?'].value_counts()[0],
                            mean1=final_result[final_result['Climate Zone']=='Snow'][sim + ' (75th %tile)'].mean(),
                            mean2=final_result[final_result['Climate Zone']=='Snow'][sim + ' (25th %tile)'].mean(),
                            std_dev1=final_result[final_result['Climate Zone']=='Snow'][sim + ' (75th %tile) Std'].mean(), 
                            std_dev2=final_result[final_result['Climate Zone']=='Snow'][sim + ' (25th %tile) Std'].mean())
    gap_sig_snow.append(pval)
    
    pval = two_sample_ttest(n1=data[(data['Median Income (75th percentile)?']==True) & (data['Climate Zone'] == 2)]['Median Income (75th percentile)?'].value_counts()[0],
                            n2=data[(data['Median Income (25th percentile)?']==True) & (data['Climate Zone'] == 2)]['Median Income (25th percentile)?'].value_counts()[0],
                            mean1=final_result[final_result['Climate Zone']=='Temperate'][sim + ' (75th %tile)'].mean(),
                            mean2=final_result[final_result['Climate Zone']=='Temperate'][sim + ' (25th %tile)'].mean(),
                            std_dev1=final_result[final_result['Climate Zone']=='Temperate'][sim + ' (75th %tile) Std'].mean(), 
                            std_dev2=final_result[final_result['Climate Zone']=='Temperate'][sim + ' (25th %tile) Std'].mean())
    gap_sig_temp.append(pval)
    
    pval = two_sample_ttest(n1=data[(data['Median Income (75th percentile)?']==True) & (data['Climate Zone'] == 3)]['Median Income (75th percentile)?'].value_counts()[0],
                            n2=data[(data['Median Income (25th percentile)?']==True) & (data['Climate Zone'] == 3)]['Median Income (25th percentile)?'].value_counts()[0],
                            mean1=final_result[final_result['Climate Zone']=='Tropical'][sim + ' (75th %tile)'].mean(),
                            mean2=final_result[final_result['Climate Zone']=='Tropical'][sim + ' (25th %tile)'].mean(),
                            std_dev1=final_result[final_result['Climate Zone']=='Tropical'][sim + ' (75th %tile) Std'].mean(), 
                            std_dev2=final_result[final_result['Climate Zone']=='Tropical'][sim + ' (25th %tile) Std'].mean())
    gap_sig_trop.append(pval)
len(sim_col)

44

In [60]:
sim_sig_clim = pd.DataFrame({'Simulation': sim_col, 
                             'Arid pval':gap_sig_arid, 
                             'Snow pval':gap_sig_snow, 
                             'Temperate pval':gap_sig_temp, 
                             'Tropical pval':gap_sig_trop})
sim_sig_clim

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
0,Baseline CUHI Day,0.000000e+00,0.0,0.000000,3.879541e-11
1,Baseline CUHI Night,0.000000e+00,0.0,0.000000,2.910117e-12
2,Baseline SUHI Day,0.000000e+00,0.0,0.000000,2.220446e-16
3,Baseline SUHI Night,0.000000e+00,0.0,0.000000,0.000000e+00
4,Albedo Management CUHI Day,0.000000e+00,0.0,0.000000,1.256684e-11
5,Albedo Management CUHI Night,0.000000e+00,0.0,0.000000,0.000000e+00
6,Albedo Management SUHI Day,0.000000e+00,0.0,0.000000,0.000000e+00
7,Albedo Management SUHI Night,0.000000e+00,0.0,0.000000,0.000000e+00
8,Targeted Albedo Management CUHI Day,0.000000e+00,0.0,0.000000,1.103420e-10
9,Targeted Albedo Management CUHI Night,0.000000e+00,0.0,0.000000,0.000000e+00


In [61]:
len(sim_sig_clim[sim_sig_clim['Arid pval'] < 0.0001])

43

In [62]:
sim_sig_clim[sim_sig_clim['Arid pval'] > 0.0001]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
40,Targeted Albedo Management/100% Afforestation ...,0.003371,0.0,0.0,0.0


In [109]:
0.003371 < 0.01

True

In [71]:
sim_sig_clim[sim_sig_clim['Arid pval'] > 0.0001]['Simulation'].iloc[0]

'Targeted Albedo Management/100% Afforestation CUHI Day'

In [63]:
len(sim_sig_clim[sim_sig_clim['Snow pval'] < 0.0001])

44

In [64]:
len(sim_sig_clim[sim_sig_clim['Temperate pval'] < 0.0001])

42

In [65]:
sim_sig_clim[sim_sig_clim['Temperate pval'] > 0.0001]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
24,Targeted 100% Afforestation CUHI Day,3.742340e-12,0.0,0.001118,0.0
26,Targeted 100% Afforestation SUHI Day,0.000000e+00,0.0,0.000789,0.0


In [111]:
0.000789 < 0.0001

False

In [112]:
0.000789 < 0.001

True

In [ ]:
sim_sig_clim[sim_sig_clim['Temperate pval'] < 0.001]

In [106]:
0.001118 < 0.01

True

In [110]:
sim_sig_clim[sim_sig_clim['Tropical pval'] > 0.0001]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
15,50% Afforestation SUHI Night,0.0,0.0,0.0,0.000958
16,100% Afforestation CUHI Day,0.0,0.0,0.0,0.003458
31,Albedo Management/50% Afforestation SUHI Night,0.0,0.0,0.0,0.050147
32,Albedo Management/100% Afforestation CUHI Day,0.0,0.0,0.0,0.236574


In [66]:
sim_sig_clim[sim_sig_clim['Temperate pval'] > 0.001]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
24,Targeted 100% Afforestation CUHI Day,3.742340e-12,0.0,0.001118,0.0


In [67]:
len(sim_sig_clim[sim_sig_clim['Tropical pval'] < 0.0001])

40

In [68]:
sim_sig_clim[sim_sig_clim['Tropical pval'] > 0.0001]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
15,50% Afforestation SUHI Night,0.0,0.0,0.0,0.000958
16,100% Afforestation CUHI Day,0.0,0.0,0.0,0.003458
31,Albedo Management/50% Afforestation SUHI Night,0.0,0.0,0.0,0.050147
32,Albedo Management/100% Afforestation CUHI Day,0.0,0.0,0.0,0.236574


In [114]:
0.000958 < 0.001

True

In [ ]:
len(sim_sig_clim[sim_sig_clim['Arid pval'] < 0.0001])

In [87]:
list(sim_sig_clim.columns)

['Simulation', 'Arid pval', 'Snow pval', 'Temperate pval', 'Tropical pval']

In [92]:
pd.DataFrame({
    'Simulation': sim_col,
    'Arid pval': np.where(sim_sig_clim['Arid pval'] < 0.0001, True, np.nan),
    'Snow pval': np.where(sim_sig_clim['Snow pval'] < 0.0001, True, np.nan),
    'Temperate pval': np.where(sim_sig_clim['Temperate pval'] < 0.0001, True, np.nan),
    'Tropical pval': np.where(sim_sig_clim['Tropical pval'] < 0.0001, True, np.nan)
})

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
0,Baseline CUHI Day,1.0,1.0,1.0,1.0
1,Baseline CUHI Night,1.0,1.0,1.0,1.0
2,Baseline SUHI Day,1.0,1.0,1.0,1.0
3,Baseline SUHI Night,1.0,1.0,1.0,1.0
4,Albedo Management CUHI Day,1.0,1.0,1.0,1.0
5,Albedo Management CUHI Night,1.0,1.0,1.0,1.0
6,Albedo Management SUHI Day,1.0,1.0,1.0,1.0
7,Albedo Management SUHI Night,1.0,1.0,1.0,1.0
8,Targeted Albedo Management CUHI Day,1.0,1.0,1.0,1.0
9,Targeted Albedo Management CUHI Night,1.0,1.0,1.0,1.0


In [ ]:
clim_zones = ['Arid', 'Snow', 'Temperate', 'Tropical']

In [78]:
0.0001, 0.001, 0.01

(0.0001, 0.001, 0.01)

In [99]:
clim_zone_pvals = pd.DataFrame({
    'Simulation': sim_col,
    'Arid pval': np.where(sim_sig_clim['Arid pval'] < 0.0001, True, np.nan),
    'Snow pval': np.where(sim_sig_clim['Snow pval'] < 0.0001, True, np.nan),
    'Temperate pval': np.where(sim_sig_clim['Temperate pval'] < 0.0001, True, np.nan),
    'Tropical pval': np.where(sim_sig_clim['Tropical pval'] < 0.0001, True, np.nan)
})

In [100]:
clim_zone_pvals[clim_zone_pvals['Simulation'].str.contains('CUHI Day')]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
0,Baseline CUHI Day,1.0,1.0,1.0,1.0
4,Albedo Management CUHI Day,1.0,1.0,1.0,1.0
8,Targeted Albedo Management CUHI Day,1.0,1.0,1.0,1.0
12,50% Afforestation CUHI Day,1.0,1.0,1.0,1.0
16,100% Afforestation CUHI Day,1.0,1.0,1.0,NaN
20,Targeted 50% Afforestation CUHI Day,1.0,1.0,1.0,1.0
24,Targeted 100% Afforestation CUHI Day,1.0,1.0,NaN,1.0
28,Albedo Management/50% Afforestation CUHI Day,1.0,1.0,1.0,1.0
32,Albedo Management/100% Afforestation CUHI Day,1.0,1.0,1.0,NaN
36,Targeted Albedo Management/50% Afforestation C...,1.0,1.0,1.0,1.0


In [101]:
clim_zone_pvals[clim_zone_pvals['Simulation'].str.contains('CUHI Night')]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
1,Baseline CUHI Night,1.0,1.0,1.0,1.0
5,Albedo Management CUHI Night,1.0,1.0,1.0,1.0
9,Targeted Albedo Management CUHI Night,1.0,1.0,1.0,1.0
13,50% Afforestation CUHI Night,1.0,1.0,1.0,1.0
17,100% Afforestation CUHI Night,1.0,1.0,1.0,1.0
21,Targeted 50% Afforestation CUHI Night,1.0,1.0,1.0,1.0
25,Targeted 100% Afforestation CUHI Night,1.0,1.0,1.0,1.0
29,Albedo Management/50% Afforestation CUHI Night,1.0,1.0,1.0,1.0
33,Albedo Management/100% Afforestation CUHI Night,1.0,1.0,1.0,1.0
37,Targeted Albedo Management/50% Afforestation C...,1.0,1.0,1.0,1.0


In [102]:
clim_zone_pvals[clim_zone_pvals['Simulation'].str.contains('SUHI Day')]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
2,Baseline SUHI Day,1.0,1.0,1.0,1.0
6,Albedo Management SUHI Day,1.0,1.0,1.0,1.0
10,Targeted Albedo Management SUHI Day,1.0,1.0,1.0,1.0
14,50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
18,100% Afforestation SUHI Day,1.0,1.0,1.0,1.0
22,Targeted 50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
26,Targeted 100% Afforestation SUHI Day,1.0,1.0,NaN,1.0
30,Albedo Management/50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
34,Albedo Management/100% Afforestation SUHI Day,1.0,1.0,1.0,1.0
38,Targeted Albedo Management/50% Afforestation S...,1.0,1.0,1.0,1.0


In [103]:
clim_zone_pvals[clim_zone_pvals['Simulation'].str.contains('SUHI Day')]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
2,Baseline SUHI Day,1.0,1.0,1.0,1.0
6,Albedo Management SUHI Day,1.0,1.0,1.0,1.0
10,Targeted Albedo Management SUHI Day,1.0,1.0,1.0,1.0
14,50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
18,100% Afforestation SUHI Day,1.0,1.0,1.0,1.0
22,Targeted 50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
26,Targeted 100% Afforestation SUHI Day,1.0,1.0,NaN,1.0
30,Albedo Management/50% Afforestation SUHI Day,1.0,1.0,1.0,1.0
34,Albedo Management/100% Afforestation SUHI Day,1.0,1.0,1.0,1.0
38,Targeted Albedo Management/50% Afforestation S...,1.0,1.0,1.0,1.0


In [104]:
clim_zone_pvals[clim_zone_pvals['Simulation'].str.contains('SUHI Night')]

,Simulation,Arid pval,Snow pval,Temperate pval,Tropical pval
3,Baseline SUHI Night,1.0,1.0,1.0,1.0
7,Albedo Management SUHI Night,1.0,1.0,1.0,1.0
11,Targeted Albedo Management SUHI Night,1.0,1.0,1.0,1.0
15,50% Afforestation SUHI Night,1.0,1.0,1.0,NaN
19,100% Afforestation SUHI Night,1.0,1.0,1.0,1.0
23,Targeted 50% Afforestation SUHI Night,1.0,1.0,1.0,1.0
27,Targeted 100% Afforestation SUHI Night,1.0,1.0,1.0,1.0
31,Albedo Management/50% Afforestation SUHI Night,1.0,1.0,1.0,NaN
35,Albedo Management/100% Afforestation SUHI Night,1.0,1.0,1.0,1.0
39,Targeted Albedo Management/50% Afforestation S...,1.0,1.0,1.0,1.0


In [115]:
final_result.head()

,Urban_name,Climate Zone,Coastal?,Baseline CUHI Day AWM,Baseline CUHI Day (25th %tile),Baseline CUHI Day (25th %tile) Std,Baseline CUHI Day (75th %tile),Baseline CUHI Day (75th %tile) Std,Baseline CUHI Day (Difference),Baseline CUHI Night AWM,Baseline CUHI Night (25th %tile),Baseline CUHI Night (25th %tile) Std,Baseline CUHI Night (75th %tile),Baseline CUHI Night (75th %tile) Std,Baseline CUHI Night (Difference),Baseline SUHI Day AWM,Baseline SUHI Day (25th %tile),Baseline SUHI Day (25th %tile) Std,Baseline SUHI Day (75th %tile),Baseline SUHI Day (75th %tile) Std,Baseline SUHI Day (Difference),Baseline SUHI Night AWM,Baseline SUHI Night (25th %tile),Baseline SUHI Night (25th %tile) Std,Baseline SUHI Night (75th %tile),Baseline SUHI Night (75th %tile) Std,Baseline SUHI Night (Difference),Baseline CUHI Day Slope,Baseline CUHI Day pval,Baseline CUHI Day SE,Baseline CUHI Night Slope,Baseline CUHI Night pval,Baseline CUHI Night SE,Baseline SUHI Day Slope,Baseline SUHI Day pval,Baseline SUHI Day SE,Baseline SUHI Night Slope,Baseline SUHI Night pval,Baseline SUHI Night SE,Albedo Management CUHI Day AWM,Albedo Management CUHI Day (25th %tile),Albedo Management CUHI Day (25th %tile) Std,Albedo Management CUHI Day (75th %tile),Albedo Management CUHI Day (75th %tile) Std,Albedo Management CUHI Day (Difference),Albedo Management CUHI Night AWM,Albedo Management CUHI Night (25th %tile),Albedo Management CUHI Night (25th %tile) Std,Albedo Management CUHI Night (75th %tile),Albedo Management CUHI Night (75th %tile) Std,Albedo Management CUHI Night (Difference),Albedo Management SUHI Day AWM,Albedo Management SUHI Day (25th %tile),Albedo Management SUHI Day (25th %tile) Std,Albedo Management SUHI Day (75th %tile),Albedo Management SUHI Day (75th %tile) Std,Albedo Management SUHI Day (Difference),Albedo Management SUHI Night AWM,Albedo Management SUHI Night (25th %tile),Albedo Management SUHI Night (25th %tile) Std,Albedo Management SUHI Night (75th %tile),Albedo Management SUHI Night (75th %tile) Std,Albedo Management SUHI Night (Difference),Albedo Management CUHI Day Slope,Albedo Management CUHI Day pval,Albedo Management CUHI Day SE,Albedo Management CUHI Night Slope,Albedo Management CUHI Night pval,Albedo Management CUHI Night SE,Albedo Management SUHI Day Slope,Albedo Management SUHI Day pval,Albedo Management SUHI Day SE,Albedo Management SUHI Night Slope,Albedo Management SUHI Night pval,Albedo Management SUHI Night SE,Targeted Albedo Management CUHI Day AWM,Targeted Albedo Management CUHI Day (25th %tile),Targeted Albedo Management CUHI Day (25th %tile) Std,Targeted Albedo Management CUHI Day (75th %tile),Targeted Albedo Management CUHI Day (75th %tile) Std,Targeted Albedo Management CUHI Day (Difference),Targeted Albedo Management CUHI Night AWM,Targeted Albedo Management CUHI Night (25th %tile),Targeted Albedo Management CUHI Night (25th %tile) Std,Targeted Albedo Management CUHI Night (75th %tile),Targeted Albedo Management CUHI Night (75th %tile) Std,Targeted Albedo Management CUHI Night (Difference),Targeted Albedo Management SUHI Day AWM,Targeted Albedo Management SUHI Day (25th %tile),Targeted Albedo Management SUHI Day (25th %tile) Std,Targeted Albedo Management SUHI Day (75th %tile),Targeted Albedo Management SUHI Day (75th %tile) Std,Targeted Albedo Management SUHI Day (Difference),Targeted Albedo Management SUHI Night AWM,Targeted Albedo Management SUHI Night (25th %tile),Targeted Albedo Management SUHI Night (25th %tile) Std,Targeted Albedo Management SUHI Night (75th %tile),Targeted Albedo Management SUHI Night (75th %tile) Std,Targeted Albedo Management SUHI Night (Difference),Targeted Albedo Management CUHI Day Slope,Targeted Albedo Management CUHI Day pval,Targeted Albedo Management CUHI Day SE,Targeted Albedo Management CUHI Night Slope,Targeted Albedo Management CUHI Night pval,Targeted Albedo Management CUHI Night SE,Targeted Albedo Management SUHI Day Slope,Targeted Albedo Management SUHI Day pval,Targeted Albedo M

In [119]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Day AWM' for x in sims]].mean()).round(2).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Day AWM,-0.59,-0.44,-0.33,-0.15
Albedo Management CUHI Day AWM,-0.66,-0.45,-0.37,-0.16
Targeted Albedo Management CUHI Day AWM,-0.60,-0.44,-0.34,-0.15
50% Afforestation CUHI Day AWM,-1.03,-0.66,-0.61,-0.39
100% Afforestation CUHI Day AWM,-1.21,-0.84,-0.83,-0.69
Targeted 50% Afforestation CUHI Day AWM,-0.63,-0.46,-0.37,-0.20
Targeted 100% Afforestation CUHI Day AWM,-0.66,-0.46,-0.38,-0.21
Albedo Management/50% Afforestation CUHI Day AWM,-1.13,-0.72,-0.70,-0.44
Albedo Management/100% Afforestation CUHI Day AWM,-1.27,-0.88,-0.88,-0.72
Targeted Albedo Management/50% Afforestation CUHI Day AWM,-0.66,-0.46,-0.37,-0.21


In [134]:
-1.27--0.59, -0.88--0.44, -0.88--0.33, -0.72--0.15

(-0.68, -0.44, -0.55, -0.57)

In [135]:
-1.21--0.59, -0.84--0.44, -0.83--0.33, -0.69--0.15

(-0.62, -0.39999999999999997, -0.49999999999999994, -0.5399999999999999)

In [136]:
-1.13--0.59, -0.72--0.44, -0.70--0.33, -0.44--0.15

(-0.5399999999999999,
 -0.27999999999999997,
 -0.36999999999999994,
 -0.29000000000000004)

In [137]:
-1.03--0.59, -0.66--0.44, -0.61--0.33, -0.39--0.15

(-0.44000000000000006,
 -0.22000000000000003,
 -0.27999999999999997,
 -0.24000000000000002)

In [120]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Night AWM' for x in sims]].mean()).round(2).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Night AWM,-0.63,-0.34,-0.16,-0.18
Albedo Management CUHI Night AWM,-0.62,-0.39,-0.21,-0.22
Targeted Albedo Management CUHI Night AWM,-0.61,-0.34,-0.17,-0.19
50% Afforestation CUHI Night AWM,-0.29,0.06,0.13,-0.19
100% Afforestation CUHI Night AWM,-0.37,0.09,0.08,-0.16
Targeted 50% Afforestation CUHI Night AWM,-0.59,-0.32,-0.14,-0.19
Targeted 100% Afforestation CUHI Night AWM,-0.60,-0.32,-0.14,-0.18
Albedo Management/50% Afforestation CUHI Night AWM,-0.15,0.06,0.14,-0.10
Albedo Management/100% Afforestation CUHI Night AWM,-0.29,0.07,0.05,-0.12
Targeted Albedo Management/50% Afforestation CUHI Night AWM,-0.55,-0.32,-0.14,-0.17


In [121]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Day AWM' for x in sims]].mean()).round(2).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Day AWM,-0.53,-1.67,-1.54,-0.73
Albedo Management SUHI Day AWM,-1.03,-1.93,-1.83,-1.24
Targeted Albedo Management SUHI Day AWM,-0.67,-1.70,-1.58,-0.89
50% Afforestation SUHI Day AWM,-2.69,-2.79,-3.00,-1.70
100% Afforestation SUHI Day AWM,-3.36,-3.34,-3.82,-2.28
Targeted 50% Afforestation SUHI Day AWM,-0.81,-1.72,-1.69,-0.91
Targeted 100% Afforestation SUHI Day AWM,-0.95,-1.76,-1.77,-0.98
Albedo Management/50% Afforestation SUHI Day AWM,-3.13,-3.39,-3.40,-1.98
Albedo Management/100% Afforestation SUHI Day AWM,-3.72,-3.81,-4.00,-2.68
Targeted Albedo Management/50% Afforestation SUHI Day AWM,-0.85,-1.76,-1.73,-0.96


In [122]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' SUHI Night AWM' for x in sims]].mean()).round(2).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline SUHI Night AWM,-0.74,-0.46,-0.26,-0.15
Albedo Management SUHI Night AWM,-0.76,-0.58,-0.39,-0.18
Targeted Albedo Management SUHI Night AWM,-0.76,-0.47,-0.28,-0.15
50% Afforestation SUHI Night AWM,-0.14,0.17,0.16,-0.04
100% Afforestation SUHI Night AWM,-0.03,0.36,0.21,-0.02
Targeted 50% Afforestation SUHI Night AWM,-0.65,-0.44,-0.23,-0.11
Targeted 100% Afforestation SUHI Night AWM,-0.64,-0.42,-0.22,-0.08
Albedo Management/50% Afforestation SUHI Night AWM,-0.31,0.09,0.10,-0.07
Albedo Management/100% Afforestation SUHI Night AWM,-0.23,0.25,0.10,-0.01
Targeted Albedo Management/50% Afforestation SUHI Night AWM,-0.67,-0.44,-0.24,-0.10


In [138]:
pd.DataFrame(final_result.groupby(['Climate Zone'])[[x + ' CUHI Day (25th %tile)' for x in sims]].mean()).round(2).T

Climate Zone,Arid,Snow,Temperate,Tropical
Baseline CUHI Day (25th %tile),-0.08,0.20,0.11,-0.16
Albedo Management CUHI Day (25th %tile),-0.17,0.13,0.06,-0.16
Targeted Albedo Management CUHI Day (25th %tile),-0.17,0.13,0.06,-0.16
50% Afforestation CUHI Day (25th %tile),-0.41,-0.04,-0.17,-0.44
100% Afforestation CUHI Day (25th %tile),-0.56,-0.15,-0.31,-0.53
Targeted 50% Afforestation CUHI Day (25th %tile),-0.41,-0.04,-0.17,-0.44
Targeted 100% Afforestation CUHI Day (25th %tile),-0.56,-0.15,-0.31,-0.53
Albedo Management/50% Afforestation CUHI Day (25th %tile),-0.54,-0.14,-0.26,-0.52
Albedo Management/100% Afforestation CUHI Day (25th %tile),-0.65,-0.20,-0.38,-0.63
Targeted Albedo Management/50% Afforestation CUHI Day (25th %tile),-0.54,-0.14,-0.26,-0.52


In [141]:
def weighted_avg(group, x):
    return np.average(group[x], weights=group['Area'])
awm_df = pd.DataFrame({'AWM': data[['Urban_name', 'CUHI Day', 'Area']].dropna().groupby('Urban_name').apply(lambda group: weighted_avg(group, 'CUHI Day'))})


In [142]:
awm_df.head()

,AWM
Urban_name,
"Aberdeen--Bel Air South--Bel Air North, MD",0.063989
"Abilene, TX",-0.097544
"Aguadilla--Isabela--San Sebastián, PR",-0.399814
"Akron, OH",-0.003030
"Albany, GA",-0.247680


In [143]:
awm_df['AWM'].mean().round(2)

-0.39

In [145]:
final_result['Baseline CUHI Day AWM'].mean().round(2)

-0.38